<a href="https://colab.research.google.com/github/jerry-mouse-2333/colab_save/blob/master/slove_tensor2GB.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from __future__ import division, print_function, unicode_literals

# Common imports
import numpy as np
import os
import sys
import h5py
import tensorflow.compat.v1 as tf
from functools import partial
tf.reset_default_graph()


from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession
config = ConfigProto()
config.gpu_options.allow_growth = True

session = InteractiveSession(config=config)
session.close()

# InteractiveSession.close(session)


%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt

#定义子图、x轴、y轴的标签大小

(plt.rcParams['axes.labelsize'],plt.rcParams['xtick.labelsize'],plt.rcParams['ytick.labelsize'])  = (14,12,12)

#定义图像显示的dpi、保存的dpi，显示的窗口大小

(plt.rcParams['figure.dpi'],plt.rcParams['savefig.dpi'],plt.rcParams['figure.figsize'])=(300,300,(80,60))


def plot_image(image, desired_shape=[128, 128]):
    
    plt.imshow(image.reshape(desired_shape), cmap="Greys", interpolation="nearest")
    plt.axis("off")

def plot_multiple_images(images, n_rows, n_cols, pad=2):
    print(images)
    images = images - images.min()  # make the minimum == 0, so the padding looks white
    print(images)
    
    w,h = images.shape[1:]
    
    image = np.zeros(((w+pad)*n_rows+pad, (h+pad)*n_cols+pad))
    
    for y in range(n_rows):
        for x in range(n_cols):
            image[(y*(h+pad)+pad):(y*(h+pad)+pad+h),(x*(w+pad)+pad):(x*(w+pad)+pad+w)] = images[y*n_cols+x]
            
    plt.imshow(image, cmap="Greys", interpolation="nearest")
    plt.axis("off")

# Where to save the figures
root_dir = "/content/drive/My Drive/result of trains"

path = os.path.join(root_dir,"images")
print(path)




#tight_layout: 使子图布满图像区域
def save_fig(fig_id, tight_layout=True):
    save_path = os.path.join(path, fig_id + ".png")
    print("Saving figure to", save_path)
    print("Saving figure", fig_id)
    os.makedirs(path, exist_ok=True)
    
#    if not os.path.exists(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)):

#        os.makedirs(os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID))
    if tight_layout:
        plt.tight_layout()
    plt.savefig(save_name, format='png', dpi=300)

# filters=[32,16,32]
filters=[32,16]
conv_kernel=3

batch_size = 32
ephochs = 10

#原始图片尺寸
width = 640
height = 480
depth = 1

#输入层
input_dim = (480,640)

#编码层输出
en_output=2

#解码层输出
output_dim = input_dim

#学习率与正则参数
learning_rate = 0.001
l2_reg = 0.0001

#散度参数
sparsity_target = 0.2
sparsity_weight = 0.2

#train_method2：全连接层
en1 = 300 #hid_1 of encoder
en2 = 150  
de1 = en2 #hid_1 of decoder
de2 = en1

def kl_divergence(p, q):
    # Kullback Leibler divergence
    return p * tf.log(p / q) + (1 - p) * tf.log((1 - p) / (1 - q))


#收集变量绘制其histogram等
tensor_list=[]

with tf.name_scope("encoder") as scope:

    def en_module(en_inputs,en_output_dim,filters,k=conv_kernel):
        x=en_inputs
        i=0
        for _ in filters:
            print(_)
            # if (i==2):
            #   x = tf.layers.conv2d(x,_,3,2,name='conv_'+str(i))
            # else:
            x = tf.layers.conv2d(x,_,3,2,padding='same',name='conv_'+str(i))
            tensor_list.append(x)
            print(x.shape)
            tf.add_to_collection('reuse', x)
            x=tf.nn.leaky_relu(x, alpha=0.01,name='act_'+str(i))
            tensor_list.append(x)
            
            tf.add_to_collection('reuse', x)
            x = tf.layers.batch_normalization(x,axis=-1,name='batch_norm_'+str(i))
            tensor_list.append(x)
            
            tf.add_to_collection('reuse', x)
            i+=1
        
        shape=x.get_shape().as_list()

        print(shape)
        flat1 = tf.layers.flatten(x,name='flattened')
        #print(flat1.shape)

        #必须使用sigmoid激活使得KL_loss!=Nan
        half_output = my_dense_layer(flat1,en_output_dim,activation = "sigmoid",name="encoder_output")

        tf.add_to_collection('reuse', half_output)

        return shape,flat1,half_output

with tf.name_scope("decoder") as scope:
  def de_module(de_inputs,depth,filters,k=conv_kernel):
    x = de_inputs
    i=0

    for _ in filters[::-1]:
      x = tf.layers.conv2d_transpose(x,_,3,2,padding='same',name='conv_tran_'+str(i+len(filters)))
      print(x.shape)
      x=tf.nn.leaky_relu(x, alpha=0.01,name='de_act_'+str(i+len(filters)))
      
      x = tf.layers.batch_normalization(x,axis=-1,name='de_batch_norm_'+str(i+len(filters)))
      i+=1
      

      #整合所有通道
    outputs_op = tf.layers.conv2d_transpose(x,depth,3,1,padding='same',activation="sigmoid",name='outputs_op')

      #观察最后的重建效果
    tf.add_to_collection('reuse', outputs_op)
    outputs=outputs_op
    return outputs

tf.compat.v1.disable_eager_execution()
inputs = tf.placeholder(tf.float32, shape=[None,input_dim[0],input_dim[1],depth])#?,128,128


#把input加入到将来要用的集合中
tf.add_to_collection('reuse', inputs)
print(inputs.shape)

#he_init = tf.contrib.layers.variance_scaling_initializer() # He initialization
he_init = tf.variance_scaling_initializer()
l2_regularizer = tf.keras.regularizers.l2(l2_reg)
my_dense_layer = partial(tf.layers.dense,activation=tf.nn.elu,
                             kernel_initializer=he_init,
                             kernel_regularizer=l2_regularizer)
#en_inputs = tf.expand_dims(inputs,axis=-1,name="encoder_input")
en_inputs=inputs

shape,flat1,half_output = en_module(en_inputs,en_output_dim=en_output,filters=filters)
print('shape_in_encoder:',shape)
print(half_output.name)


decode_input = my_dense_layer(half_output,np.prod(flat1.shape[1:]),name="decoder_input")#现为（？，32*32*8）
#print(decode_input)
decode_input_reshape= tf.reshape(decode_input,shape=(-1,shape[1],shape[2],shape[3]))#how to reshape it to(?,32,32,8)

outputs = de_module(decode_input_reshape,depth=depth,filters=filters)
print(outputs)
print(outputs.name)

encode_mean = tf.reduce_mean(half_output, axis=0)

with tf.name_scope("loss") as scope:
    kl=kl_divergence(sparsity_target, encode_mean)

    sparsity_loss = tf.reduce_sum(kl_divergence(sparsity_target, encode_mean))
    reconstruction_loss = tf.reduce_mean(tf.square(outputs - en_inputs))
    loss = reconstruction_loss + sparsity_weight * sparsity_loss
    #tensor_list.append(loss)
optimizer = tf.train.AdamOptimizer(learning_rate)
training_op = optimizer.minimize(loss)

tf.summary.scalar("loss",loss)
# 用来显示标量信息，常用于画出loss、accuracy曲线
print(tensor_list)
for i in range(len(tensor_list)):
    name=tensor_list[i]
    #print(name.dtype)
    tf.summary.histogram(str(name),name) 
# 用来收集tensor，可显示出训练过程中变量的分布情况


init = tf.global_variables_initializer()
saver = tf.train.Saver() # not shown in the book


summary_save_dir = os.path.join(root_dir,"models")
print(summary_save_dir)

os.makedirs(summary_save_dir, exist_ok=True)
print(summary_save_dir )
writer= tf.summary.FileWriter(summary_save_dir,tf.get_default_graph())
#获取当前默认的保存图，并在tensorboard graph中展示


merge_op = tf.summary.merge_all()
#执行所有summary操作
print("end of before")

data_path = '/content/drive/Shared drives/shared/train_gray.h5'
h5_file=h5py.File(data_path,'r')

X_train=np.array(h5_file['X_train'])[0:5000]
# [0:5000]
#X_test=np.array(h5_file['X_test'])
print(X_train.shape)

(None, 480, 640, 1)
32
Instructions for updating:
Use `tf.keras.layers.Conv2D` instead.
Instructions for updating:
Please use `layer.__call__` method instead.
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
(None, 240, 320, 32)
Instructions for updating:
Use keras.layers.BatchNormalization instead.  In particular, `tf.control_dependencies(tf.GraphKeys.UPDATE_OPS)` should not be used (consult the `tf.keras.layers.BatchNormalization` documentation).
16
(None, 120, 160, 16)
[None, 120, 160, 16]
Instructions for updating:
Use keras.layers.Flatten instead.
Instructions for updating:
Use keras.layers.Dense instead.
shape_in_encoder: [None, 120, 160, 16]
encoder_output/Sigmoid:0
Instructions for updating:
Use `tf.keras.layers.Conv2DTranspose` instead.
(None, 240, 320, 16)
(None, 480, 640, 32)
Tensor("outputs_op/Sigmoid:0", shape=(None, 480, 640, 1), dtype=float32)
outputs_op/Sigmoid:0
[<tf.Tensor 'conv_0/BiasAdd:0' shape=(None, 240, 320, 32) dtype=float32>, <t

In [3]:
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.8) 
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True, gpu_options=gpu_options))
def get_batch(dataset,ephochs,batch_size):
    print('o')
    with tf.variable_scope('shabi',reuse=tf.AUTO_REUSE):
      d_plhd = tf.placeholder(dtype=tf.float32, shape=(None,480,640))  #定义一个placeholder  
      dl = tf.get_variable('d1',[5000,480,640])   #再定义一个变量
      print(dl.name)
    # print('h')
      with tf.Session() as sess:
        sess.run(tf.global_variables_initializer())
        sess.run(tf.local_variables_initializer())
        sess.run(dl.assign(d_plhd), {d_plhd: dataset})
      # print sess.run(tf.reduce_sum(w))++++++++
    # print('y')
        input_queue = tf.train.slice_input_producer([dl],num_epochs=ephochs,shuffle=True)
    # with tf.Session() as sess:
    #     sess.run(tf.global_variables_initializer())
    #     sess.run(tf.local_variables_initializer())
    #     coord = tf.train.Coordinator()
    #     threads = tf.train.start_queue_runners(sess, coord)
        
    #     try:
    #       for _ in range(1):
    #         print(_)
    #         print(dataset.shape)
    #         sess.run(input_queue_op,feed_dict={dataset_0:dataset})

    #         # print('ok')
    #     except tf.errors.OutOfRangeError:
    #         print("done")
    #     finally:
    #         coord.request_stop()
    #     coord.join(threads)
        print('ohye')
        X_batch = tf.train.batch(input_queue, batch_size=batch_size, num_threads=1, capacity=64)

    print(X_batch.shape)
    return X_batch
X_batch= get_batch(X_train,ephochs,batch_size)
with tf.Session() as sess:
    init.run()
    sess.run(tf.initialize_all_variables())
    sess.run(tf.local_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess, coord)
    try:
        for epoch in range(ephochs):
            n_batches = X_train.shape[0] // batch_size
            print(n_batches)
            for iteration in range(n_batches):
                # print("\r{}%".format(100 * iteration // n_batches), end="") # not shown in the book
                sys.stdout.flush()
                x_batch=sess.run(X_batch)
                x_batch = np.expand_dims(x_batch,axis=-1)
                # print(x_batch.shape)
                sess.run(training_op, feed_dict={inputs: x_batch})
                result=sess.run(merge_op, feed_dict={inputs: x_batch})
                writer.add_summary(result,iteration)
                
            reconstruction_loss_val, sparsity_loss_val, loss_val = sess.run([reconstruction_loss,sparsity_loss, loss], feed_dict={inputs: x_batch})
            #print(sess.run(kl,feed_dict={X: x_batch}))
            print("\r{}".format(epoch), "Train MSE:", reconstruction_loss_val, "\tSparsity loss:",sparsity_loss_val, "\tTotal loss:", loss_val)           # not shown
              
    except tf.errors.OutOfRangeError:
        print("done")
    finally:
        coord.request_stop()
    coord.join(threads)
    
    save_dir = '/content/drive/My Drive/result of trains'
    print(save_dir)
    save_path = os.path.join(save_dir,"my_model_gray_5000_sparse_2.ckpt")

    saver.save(sess, save_path) 
    print('ok')


Device mapping:
/job:localhost/replica:0/task:0/device:XLA_CPU:0 -> device: XLA_CPU device
/job:localhost/replica:0/task:0/device:XLA_GPU:0 -> device: XLA_GPU device
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0

o
shabi/d1:0
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(tuple(tensor_list)).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.range(limit).shuffle(limit).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epo

In [2]:
from tensorflow.python.client import device_lib

# 列出所有的本地机器设备
local_device_protos = device_lib.list_local_devices()
# 打印
#     print(local_device_protos)

# 只打印GPU设备
[print(x) for x in local_device_protos if x.device_type == 'GPU']

name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 15701463552
locality {
  bus_id: 1
  links {
  }
}
incarnation: 1545449782497483954
physical_device_desc: "device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0"



[None]

In [6]:
root_dir = "/content/drive/My Drive/result of trains"
sub_dir = "/My Drive/result of trains"
# path = os.path.join(root_dir,sub_dir,"images")
path = os.path.join(root_dir,"models")
print(path)

/content/drive/My Drive/result of trains/models
